In [6]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
import shapefile
import geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from fsUtils import isFile
from ioUtils import showSize, saveJoblib
from geoUtils import *
from geospatialUtils import saveGeoData, getBB, rmZipDir, getShapeFileInfo, getShapeIter, getGeos
import pickle
from glob import glob
from os.path import basename,splitext,join
from collections import Counter

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-11-15 11:37:53.220897


In [7]:
# Global Params
prec=7
mainkey="OpenScience"
basedir="/Users/tgadf/Downloads/OpenScience"

In [25]:
def parseOS(dirval, prec):
    from os.path import basename, dirname, join, splitext
    from glob import glob
    from collections import Counter
    useUnique = False
    osmpath   = dirval
    osms      = [splitext(basename(x))[0] for x in glob(join(osmpath, "*.shp"))]
    if len(osms) == 0:
        osms      = [splitext(basename(x))[0] for x in glob(join(osmpath, "*", "*.shp"))]
        if len(osms) > 0:
            osmpath = glob(join(osmpath, "*"))[0]
    if len(osms) == 0:
        print("There are no shape files in {0}".format(osmpath))
        return None
    print("Directory: {0}".format(osmpath))
    print("      OSM: {0}".format(osms))
    blgCntr   = Counter()
    nameCntr  = Counter()
    nameLookup = {}
    
    
    shapeData   = {}
    geoShapeMap = {}
    ngeos     = 0
    totalgeos = 0
    show      = False
    cntr      = Counter()
        
    for osm in osms:
        print(osm)
        shapeval = osm
        
        files = glob(join(basedir, "{0}-{1}*".format(osm, prec)))
        if len(files) > 0:
            print("Already processed this one...")
            #continue

            
        ## Standard Shape File Reader
        shapedir = join(osmpath, osm)
        sf       = getShapeIter(shapedir)
        retval   = getShapeFileInfo(sf)
        fields   = retval["Fields"]
        Nshapes  = retval["Num"]

        start,cmt = clock("\n\nAnalyzing {0}\t{1}".format(shapeval, Nshapes))
        if show:
            print("Fields -> {0}".format(fields))

            
        ## Interate over shapes
        irec = -1
        for shapeRec in sf.iterShapeRecords():
            irec += 1
            if irec % 2500 == 0 and irec > 0:
                if True:
                    print("Processed {0}/{1} records. Found {2} geos so far...".format(irec, Nshapes, ngeos))

            record = shapeRec.record
            shape  = shapeRec.shape
            points = shape.points
            
            fclass = None
            fclass = osm
            if False:
                if osm.startswith("Arenas") is True:
                    fclass = record[19]
                    if fclass.startswith("Elementary") is True:
                        fclass="PublicSchool"
                    else:
                        raise ValueError("Don't understand {0} for {1}".format(fclass, osm))
                    geoid  = fclass
                if osm == "PublicSchools":
                    fclass = record[19]
                    if fclass.startswith("Elementary") is True:
                        fclass="PublicSchool"
                    else:
                        raise ValueError("Don't understand {0} for {1}".format(fclass, osm))
                    geoid  = fclass
                if osm == "PrivateSchools":
                    fclass = record[19]
                    if fclass.startswith("Elementary") is True:
                        fclass="PrivateSchool"
                    else:
                        raise ValueError("Don't understand {0} for {1}".format(fclass, osm))
                    geoid  = fclass
                if osm == "CollegesUniversities":
                    fclass = record[18]
                    if fclass.startswith("Colleges") is True:
                        fclass="University"
                    elif fclass.startswith("Junior") is True:
                        fclass="JuniorCollege"
                    elif fclass.find("Trade Schools") != -1:
                        fclass="Trade"
                    elif fclass.find("Cosmetology") != -1:
                        fclass="Cosmetology"
                    elif fclass.startswith("Business") is True:
                        fclass="Business"
                    elif fclass.find("Fine Arts") != -1:
                        fclass="Arts"
                    elif fclass.startswith("Computer") is True:
                        fclass="Computer"
                    elif fclass.startswith("Flight") is True:
                        fclass="Flight"
                    else:
                        raise ValueError("Don't understand {0} for {1}".format(fclass, osm))

                    geoid  = fclass
                    enroll = record[30] ## Not used, but could


                if fclass is None:
                    #fclass = record[1]
                    print(record)
                    print("Don't know about {0}".format(osm))
                    break
                
            shapeData[geoid] = {"Name": fclass, "Record": irec}
            cntr[geoid] += 1
            #if irec > 1000: 1/0

            ## Get Geohashs
            geos  = getGeos(shape, prec=prec, returnKeys=True)
            geoShapeMap[geoid] = geos
            ngeos += len(geos)

        print("Finished: {0}".format(cntr.most_common(100)))
        print("\n")
            
        if len(shapeData) > 0:
            #for geoid in geoShapeMap.keys():
            #    geoShapeMap[geoid] = list(geoShapeMap[geoid].keys())
            print("Found {0} geos from {1}".format(ngeos, shapeval))
            saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, join(basedir, "{0}-{1}".format(osm, prec)))
                ## Record

In [26]:
import zipfile
from glob import glob
from os import mkdir
from os.path import splitext, basename, dirname, join, exists
zipfiles = glob(join(basedir, "*.zip"))
for zipname in zipfiles:
    osdir = dirname(zipname)
    name     = splitext(basename(zipname))[0]
    dirval   = join(osdir, name) 
    if exists(dirval):
        print("Directory {0} already exists".format(dirval))
        rmZipDir(dirval)
        continue
    try:
        mkdir(dirval)
    except:
        pass
    zip_ref = zipfile.ZipFile(zipname, 'r')
    print("Extract {0}".format(zipname))
    zip_ref.extractall(dirval)
    zip_ref.close()
    
    parseOS(dirval, prec)
    
    rmZipDir(dirval)

Extract /Users/tgadf/Downloads/OpenScience/AgriculturalMin.zip
Directory: /Users/tgadf/Downloads/OpenScience/AgriculturalMin/AgriculturalMineralOperations
      OSM: ['AgriculturalMineralOperations']
AgriculturalMineralOperations
Already processed this one...
Current Time is Thu Nov 15, 2018 12:16:51 for 

Analyzing AgriculturalMineralOperations	None


UnboundLocalError: local variable 'geoid' referenced before assignment

In [ ]:
import pickle
prec=7
from glob import glob
from os.path import basename,splitext, join
from collections import Counter

#mtypes   = set([splitext(basename(x))[0].split('-')[1] for x in vals])
mtypes  = ["traffic", "pofw", "pois", "transport", "buildings"]
mtypes  = ["buildings"]
mtypes  = ["pofw"]
mtypes  = ["traffic"]
mtypes  = ["transport"]
mtypes  = ["pois"]
mtypes  = ["traffic", "pofw", "pois", "transport"]

print("Mtypes --> {0}".format(mtypes))
for mtype in mtypes:
    if mtype in ["pofw","traffic","transport","pois"]:
        vals    = glob(join(basedir, "*-{0}-{1}-geos.p".format(mtype, prec)))        
        states  = set([splitext(basename(x))[0].split('-{0}'.format(mtype))[0] for x in vals])
    else:
        vals    = glob(join(basedir, "*-{0}-geos.p".format(prec)))
        states  = set([splitext(basename(x))[0].split('-gis_osm')[0] for x in vals])
    print("States --> {0}".format(states))

    print("============== {0} ==============".format(mtype))
    fnames = letters[:1]
    for let in range(len(fnames)):
        ngeosMtypes = 0
        ngeos = 0
        geomap  = {}
        records = {}
        cntr    = Counter()
        for istate,state in enumerate(states):
            if istate % len(fnames) != let:
                continue
            base = "{0}*{1}*{2}".format(state, mtype, prec)
            recfiles = glob(join(basedir, "{0}-data.p".format(base)))
            geofiles = glob(join(basedir, "{0}-geos.p".format(base)))

            #print("  Found {0} and {1} rec/geo files for {2}".format(len(recfiles), len(geofiles), mtype))
            for i in range(len(geofiles)):
                recfile = recfiles[i]
                geofile = geofiles[i]

                try:
                    recdata = pickle.load(open(recfile, "rb"))
                    geodata = pickle.load(open(geofile, "rb"))
                    #print("Opened {0}".format(recfile))
                    #print("Opened {0}".format(geofile))
                except:
                    print("Could not open {0}".format(geofile))
                    1/0
                    continue

                #print(geodata.keys())
                #continue
                ngeos = 0
                for geoid,geos in geodata.items():
                    ngeos += len(geos)
                    ngeosMtypes += len(geos)
                    for geo in geos:
                        if geomap.get(geo) is None:
                            geomap[geo] = Counter()
                        #print(geo,mtype,geoid)
                        #print(geoid)
                        #print(recdata.keys())
                        if recdata.get(geoid) is not None:
                            try:
                                #print(geoid)
                                #print(recdata[geoid])
                                #name = recdata[geoid]['Name']
                                #print(name)
                                name = geoid
                            except:
                                name = geoid
                                #print(name)
                        else:
                            name = geoid
                        cntr[name] += 1
                        geomap[geo][name] += 1

                for geoid,rec in recdata.items():
                    records[geoid] = rec['Name']
                    break

            print("{0: <20}\t{1: <20}\t{2: <12}\t{3: <12}\t---> {4} <---".format(mtype,state,ngeos,ngeosMtypes,len(geomap)))            
        print("{0: <20}\t{1: <20}\t{2: <12}\t{3: <12}\t---> {4} <---".format(mtype,"","",ngeosMtypes,len(geomap)))

        for k,v in cntr.most_common():
            k = str(k)
            if k.find(" ") != -1 or k.find(";") != -1 or k.find("_") != -1 or k.find(",") != -1 or k[0].isupper():
                print("buildingConv[\"{0}\"] = \"{1}\"".format(k, k))
                found = True
                    
        for geo in geomap.keys():
            geomap[geo] = dict(geomap[geo])
        

        fname="geomap-{0}-{1}OSM.p".format(prec, mtype) #, fnames[let])
        print("Writing {0}".format(fname))
        pickle.dump(geomap,  open(fname, "wb"))

        fname="georec-{0}-{1}OSM.p".format(prec, mtype) #, fnames[let])
        print("Writing {0}".format(fname))
        pickle.dump(records,  open(fname, "wb"))

In [ ]:
prec = 7
from glob import glob
from os.path import join, basename
show = False
for zipfilename in zipfiles:
    extractDir = unZipFile(zipfilename, test=False)
    shapeDir   = extractDir
    if True:
        try:
            shapeDir   = glob(join(extractDir, '*'))[0]
        except:
            continue
    
    officialName = basename(shapeDir)
    shapeval     = officialName
    catID        = officialName
    sf = getShapeData(shapeDir)
    if sf is None:
        print("Shapefile data is None!")
        rmZipDir(zipfilename)
        continue
    else:
        print("  Shapefile data is ready to use")
        
        
    fields      = sf.fields
    shapeData   = {}
    geoShapeMap = {}
    Nshapes     = len(sf.shapes())
    totalgeos   = 0

    start,cmt = clock("Analyzing {0}\t{1}".format(shapeDir, Nshapes))
    if show:
        print("\n\nFields -> {0}".format(fields))

    irec  = -1
    ngeos = 0
    
    try:
        sfIter = sf.iterShapeRecords()
    except:
        print("Could not iterate over shape records")
        rmZipDir(zipfilename)
        continue
    
    
    useLinear = False
    for shapeRec in sfIter:
        irec += 1
        record = shapeRec.record
        shape  = shapeRec.shape
        points = shape.points
        if len(points) == 1 or useLinear is True:
            useLinear = True
            geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False)
        else:
            geos = getInitGeo(shape)
            geos = addShapeGeos(irec, Nshapes, shape, prec, geos, debug=False)
            if geos is None:
                useLinear = True
                geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False)

        ngeos += len(geos)
        if shapeData.get(catID) is None:
            shapeData[catID] = {}
            shapeData[catID][catID]   = {"Name": officialName}
            geoShapeMap[catID] = {}
            geoShapeMap[catID][catID] = set()
        for geo in geos:
            geoShapeMap[catID][catID].add(geo)

    if ngeos > 0:
        print("Found {0} geos from {1}".format(ngeos, shapeval))
        saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, "OpenScience/{0}-{1}".format(shapeval, 7))
        
    rmZipDir(zipfilename)

In [ ]:
def getGeoRecData(basedir, vals, prec):
    geomap  = {}
    records = {}    
    totalgeos = 0
    for mtype in vals:
        try:
            recdata = pickle.load(open("{0}/{1}-{2}-data.p".format(basedir, mtype, prec), "rb"))
            geodata = pickle.load(open("{0}/{1}-{2}-geos.p".format(basedir, mtype, prec), "rb"))
        except:
            print("Could not open files with base {0}/{1}-{2}".format(basedir, mtype, prec))
            continue
        #print("--> {0} {1}".format(len(recdata), len(geodata)))
        ngeos = 0
        for geoid,dummy in geodata.items():
            for gdummy, geos in dummy.items():
                ngeos += len(geos)
                totalgeos += ngeos
                for geo in geos:
                    if geomap.get(geo) is None:
                        geomap[geo] = {}
                    geomap[geo][mtype] = geoid

        for geoid,rec in recdata.items():
            records[geoid] = mtype
            #records[geoid] = rec['Name']
            break

        print("{0}\t{1}\t---> {2} <---".format(mtype,ngeos,len(geomap)))
        
    print("Found {0} total geohashs".format(totalgeos))
    return geomap, records

In [ ]:
venues   = ['Arenas_NBA', 'Arenas_NCAA_Div_1_Basketball', 'Arenas_NHL', 'Arenas_WNBA', 'Stadiums_MLB', 'Stadiums_NCAA_Div_1_Football', 'Stadiums_NFL', 'State_Fairgrounds', 'Raceways_NASCAR', 'Tracks_Horses', 'Tracks_IRL']
industry = ['AgriculturalMineralOperations', 'ConstructionMineralsOperations', 'CrushedStoneOpsUS', 'FerrousMetalMines', 'SandGravelMineralOps', 'FerrousMetalProcessingPlants', 'Mines', 'Nonferrous_Metal_Mines', 'RefractAbrasiveOtherIndMinOps', 'Uranium_Vanadium', 'MiscellaneousIndMineralOps']
rail     = ['Rail_Points', 'Railroad', 'RRBRIDGES']
car      = ['Bridges', 'TransitLine', 'TransitStation', 'Weigh_in_Motion_Stations', 'Automatic_Traffic_Counters', 'Tunnels']
college  = ['CollegesUniversities']
schools  = ['PrivateSchools', 'PublicSchools']
commerce = ['Banks_Main_Offices', 'CorporateHeadquarters', 'CreditUnionsHQ', 'Pharmacies', 'UPSLocations', 'DHLLocations', 'FDIC_InsuredBanks', 'FedExLocations']
terminal = ['CruiseLineTerminals', 'Ports', 'Navigable_Waterway_Nodes', 'Private_NonRetail_Shipping', 'Inland_Waterway_MM', 'Intermodal_Terminal_Facilities', 'Airports_Heliports']

mtypes = {"Venues": venues, "Industry": industry, "Rail": rail, "Car": car, "College": college, "Schools": schools, "Commerce": commerce, "Terminal": terminal}

vals    = glob("OpenScience/*-{0}-geos.p".format(prec))
vals    = [splitext(basename(x))[0].split('-')[0] for x in vals]
print(len(vals))
vals    = [x for x in vals if x not in venues]
print(len(vals))
vals    = [x for x in vals if x not in industry]
print(len(vals))
vals    = [x for x in vals if x not in rail]
print(len(vals))
vals    = [x for x in vals if x not in car]
print(len(vals))
vals    = [x for x in vals if x not in college]
print(len(vals))
vals    = [x for x in vals if x not in schools]
print(len(vals))
vals    = [x for x in vals if x not in commerce]
print(len(vals))
vals    = [x for x in vals if x not in terminal]
print(vals)

In [ ]:
for mtype, vals in mtypes.items():
    import pickle
    prec=7
    from glob import glob
    from os.path import basename,splitext
    #vals    = glob("OpenScience/*-{0}-geos.p".format(prec))
    #vals    = [splitext(basename(x))[0].split('-')[0] for x in vals]
    print(vals)
    geomap, records = getGeoRecData("OpenScience", vals, prec)
    print(geomap)
    print(records.keys())
    1/0
    
    fname="geomap-{0}-{1}.p".format(prec, mtype)
    print("Writing {0}".format(fname))
    pickle.dump(geomap,  open(fname, "wb"))

    fname="georec-{0}-{1}.p".format(prec, mtype)
    print("Writing {0}".format(fname))
    pickle.dump(records,  open(fname, "wb"))